In [1]:
from auto_encoder import AutoEncoder
from masked_dataset import CIFAR10DataModule, IMDBDataModule
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import os
from pytorch_lightning.loggers import TensorBoardLogger
%load_ext autoreload
%autoreload 2

In [2]:
cifar10 = CIFAR10DataModule()

In [3]:
in_channel_size = 3
num_epochs = 10
model = AutoEncoder(in_channel_size=3)

trainer = pl.Trainer(
    default_root_dir=os.getcwd(),
    logger=TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs"),
    auto_lr_find=True, 
    auto_scale_batch_size=True, 
    accelerator="auto",
    log_every_n_steps=1, 
    max_epochs=50, 
    min_epochs=10,
    callbacks=[
        ModelCheckpoint(save_last=True),
        LearningRateMonitor("epoch"),
        EarlyStopping(monitor="val_loss", mode="min", patience=5)
    ])
trainer.logger._log_graph = True

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
# Basic hyperparameter tuning
trainer.tune(model, cifar10)

Found cached dataset cifar10 (/Users/barissen/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Run the training
trainer.fit(model, cifar10)